In [174]:
import numpy as np
from hmmlearn import hmm
from hmmlearn.base import ConvergenceMonitor
from tqdm import tqdm

In [4]:
from requests import get as _get

BASE_URL = 'http://localhost:8080'


def get(url: str) -> list:
    l = _get(f'{BASE_URL}/{url}').json()
    print(l[0])
    print(f'Fetched {len(l)} items')
    return l

In [5]:
detections = sorted(get('detection'), key=lambda x: x["timestamp"])

{'id': 479017, 'batonId': 15, 'stationId': 9, 'rssi': -95, 'battery': 92.0, 'uptimeMs': 554087, 'remoteId': 1, 'timestamp': 1650975012000}
Fetched 2401946 items


In [6]:
batons = get('baton')

{'id': 2, 'name': 'O', 'mac': '5a:45:55:53:00:00'}
Fetched 22 items


In [7]:
baton_switchovers = sorted(get('batonswitchover'), key=lambda x: x["timestamp"])

{'id': 1, 'teamId': 1, 'previousBatonId': None, 'newBatonId': 23, 'timestamp': 1651012254359}
Fetched 73 items


In [8]:
stations = get('station')

{'id': 2, 'name': 'station 1', 'distanceFromStart': 11.0, 'isBroken': False, 'url': 'http://172.12.50.101:8000'}
Fetched 7 items


In [9]:
teams = get('team')

{'id': 1, 'name': 'HILOK', 'batonId': 18}
Fetched 17 items


In [251]:
team_detections: dict[int, list] = {team["id"]: [] for team in teams}
team_by_id: dict[int, dict] = {team["id"]: team for team in teams}
baton_team: dict[int, dict] = {}

switchover_index = 0
for detection in detections:
    while switchover_index < len(baton_switchovers) and baton_switchovers[switchover_index]["timestamp"] < detection[
        "timestamp"]:
        switchover = baton_switchovers[switchover_index]
        baton_team[switchover["newBatonId"]] = team_by_id[switchover["teamId"]]
        switchover_index += 1

    if detection["batonId"] in baton_team:
        if detection["rssi"] > -75:
            current_detections = team_detections[baton_team[detection["batonId"]]["id"]]
            if len(current_detections) > 0 and current_detections[-1]["timestamp"] == detection["timestamp"]:
                if current_detections[-1]["rssi"] < detection["rssi"]:
                    current_detections[-1] = detection
            else:
                current_detections.append(detection)

In [194]:
print({k: len(v) for k, v in team_detections.items()})

{1: 35803, 3: 31501, 4: 27289, 5: 34107, 8: 34316, 9: 29657, 10: 32160, 11: 30749, 12: 31462, 13: 29959, 14: 30143, 17: 24776, 7: 30979, 22: 36676, 6: 31973, 2: 29907, 16: 28629}


In [265]:
station_to_emission = {v: k for k, v in enumerate([station["id"] for station in stations])}
data = np.array([[station_to_emission[detection["stationId"]] for detection in team_detections[1]]])
transmat = np.array([
    [0.8, 0.1, 0., 0., 0., 0., 0.1],
    [0.1, 0.8, 0.1, 0., 0., 0., 0.],
    [0., 0.1, 0.8, 0.1, 0., 0., 0.],
    [0., 0., 0.1, 0.8, 0.1, 0., 0.],
    [0., 0., 0., 0.1, 0.8, 0.1, 0.],
    [0., 0., 0., 0., 0.1, 0.8, 0.1],
    [0.1, 0., 0., 0., 0., 0.1, 0.8]
])
emissionprob = np.array([
    [0.76, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04],
    [0.04, 0.76, 0.04, 0.04, 0.04, 0.04, 0.04],
    [0.04, 0.04, 0.76, 0.04, 0.04, 0.04, 0.04],
    [0.04, 0.04, 0.04, 0.76, 0.04, 0.04, 0.04],
    [0.04, 0.04, 0.04, 0.04, 0.76, 0.04, 0.04],
    [0.04, 0.04, 0.04, 0.04, 0.04, 0.76, 0.04],
    [0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.76],
])

model = hmm.CategoricalHMM(n_components=7, n_iter=1000, init_params="cs")
model.transmat_ = transmat.copy()
model.emissionprob_ = emissionprob.copy()
model.monitor_ = ConvergenceMonitor(model.monitor_.tol, model.monitor_.n_iter, model.monitor_.verbose)

model.fit(data)
model.monitor_

ConvergenceMonitor(
    history=[-19701.630445676125, -11575.934085030913, -11145.653186318828, -11105.002855572995, -11095.971895066616, -11093.064062373169, -11091.935464392152, -11091.452552853838, -11091.233446249087, -11091.129551025231, -11091.078459015698, -11091.05253373978, -11091.039005785087, -11091.031762857881],
    iter=14,
    n_iter=1000,
    tol=0.01,
    verbose=False,
)

In [253]:
model.startprob_

array([0.00000000e+000, 0.00000000e+000, 1.49383360e-097, 1.00000000e+000,
       6.25797576e-022, 8.33325223e-143, 0.00000000e+000])

In [254]:
model.sample(30)

(array([[3],
        [3],
        [3],
        [4],
        [4],
        [4],
        [4],
        [5],
        [6],
        [6],
        [6],
        [6],
        [0],
        [1],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [3],
        [4],
        [5],
        [5],
        [5],
        [6],
        [6],
        [6],
        [0],
        [1]]),
 array([3, 3, 3, 4, 4, 4, 4, 5, 6, 6, 6, 6, 0, 1, 2, 2, 2, 2, 2, 2, 3, 4,
        5, 5, 5, 6, 6, 6, 0, 1]))

In [255]:
E, Z = model.sample(30)

for e, v in zip(E, Z):
    print(e, v)

[3] 3
[3] 3
[4] 4
[4] 4
[5] 5
[5] 5
[5] 5
[5] 5
[6] 6
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[1] 1
[1] 1
[2] 2
[2] 2
[2] 2
[3] 3
[3] 3
[3] 4
[4] 4
[5] 5
[5] 5
[5] 5
[5] 5
[6] 6
[6] 6
[6] 6


In [206]:
models = []
for i in tqdm(range(100)):
    m = hmm.CategoricalHMM(n_components=7, n_iter=10, init_params="mcs")
    m.transmat_ = transmat.copy()
    m.fit(data)
    models.append((m.score(data), m))

best = sorted(models, key=lambda x: -x[0])[:10]
for s, m in best:
    m.n_iter = 100
    m.fit(data)
    print(m.emissionprob_)

100%|██████████| 100/100 [01:10<00:00,  1.42it/s]
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'


[[3.20880524e-084 3.09951943e-083 1.75132330e-005 9.99981565e-001
  9.21676619e-007 5.11612231e-033 8.29983577e-083]
 [5.45594498e-089 7.22318454e-226 3.40513163e-135 5.02410082e-003
  9.90388377e-001 4.58752183e-003 8.93265739e-030]
 [2.60159186e-003 8.70399470e-045 5.43147811e-004 2.52007531e-004
  1.18288991e-003 9.89292489e-001 6.12787375e-003]
 [3.45218653e-002 2.44240195e-004 5.35134296e-007 2.40368654e-174
  1.25031151e-150 2.76126757e-003 9.62472092e-001]
 [9.93231958e-001 1.90909669e-003 1.21867341e-098 1.16169900e-201
  0.00000000e+000 5.46336741e-114 4.85894577e-003]
 [1.30249317e-002 9.78762157e-001 8.21291112e-003 1.66495225e-062
  0.00000000e+000 2.46059975e-263 2.64175337e-010]
 [1.86245294e-003 4.52003009e-003 9.90428276e-001 2.51596298e-003
  1.26417078e-090 4.69131980e-171 6.73278243e-004]]


Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'


[[3.45271984e-002 2.46218045e-004 3.30914934e-006 1.57703999e-169
  4.17974606e-147 2.57328462e-003 9.62649990e-001]
 [2.60902964e-003 1.10885150e-044 5.40547436e-004 2.50914124e-004
  1.04069660e-003 9.89262432e-001 6.29638002e-003]
 [1.09620844e-103 1.02156959e-254 1.84550385e-150 5.02444744e-003
  9.90246631e-001 4.72892162e-003 1.25741821e-033]
 [5.32532089e-083 1.47290888e-080 1.16363288e-004 9.99883441e-001
  1.95676602e-007 1.74204500e-030 1.37315217e-082]
 [1.84173975e-003 4.02518845e-003 9.91024141e-001 2.43482385e-003
  1.57808250e-090 1.49642450e-182 6.74106685e-004]
 [1.30726465e-002 9.78050354e-001 8.87699895e-003 1.44469002e-059
  0.00000000e+000 7.74136946e-281 6.20914150e-010]
 [9.93274359e-001 1.86652932e-003 5.68700648e-113 2.84654514e-210
  0.00000000e+000 8.46280484e-110 4.85911126e-003]]


Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'


...

[[3.45269322e-002 2.44120601e-004 9.78919856e-008 6.45607902e-154
  1.36651283e-137 2.60376670e-003 9.62625083e-001]
 [9.93227177e-001 1.91474937e-003 9.03962698e-101 4.00282194e-210
  0.00000000e+000 4.71978319e-117 4.85807343e-003]
 [1.30037130e-002 9.80153089e-001 6.84319750e-003 1.74276539e-063
  0.00000000e+000 1.73699599e-274 1.72656278e-010]
 [1.89641697e-003 5.54711946e-003 9.89645385e-001 2.23889254e-003
  8.54216210e-095 5.56815943e-216 6.72186329e-004]
 [7.21218838e-079 4.26376504e-077 3.36011625e-004 9.99663987e-001
  1.39304085e-009 2.16379625e-033 7.05096039e-078]
 [7.26588675e-107 2.07714226e-244 1.90512014e-139 5.02495939e-003
  9.90231518e-001 4.74352294e-003 3.30310976e-035]
 [2.60807956e-003 1.60979506e-044 5.43565026e-004 2.50682587e-004
  1.02614513e-003 9.89301923e-001 6.26960430e-003]]


Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'


[[9.93258671e-001 1.87515623e-003 5.15151077e-108 9.34006635e-207
  0.00000000e+000 1.36286343e-106 4.86617291e-003]
 [1.30611570e-002 9.78365036e-001 8.57380651e-003 2.10181755e-061
  0.00000000e+000 1.46889810e-260 3.99368231e-011]
 [1.84963462e-003 4.25076832e-003 9.90699526e-001 2.52641572e-003
  3.53682756e-089 2.16076258e-166 6.73654988e-004]
 [2.23406116e-081 1.24871796e-081 7.23568258e-006 9.99992189e-001
  5.75743435e-007 3.43617811e-024 1.66699504e-083]
 [7.94257967e-101 1.78392815e-242 6.31774484e-151 5.02576373e-003
  9.90797464e-001 4.17677229e-003 4.85587651e-028]
 [2.57701634e-003 9.00313464e-055 8.21922925e-009 2.55220486e-004
  1.59836495e-003 9.90075620e-001 5.49377034e-003]
 [3.44703787e-002 2.45132553e-004 5.85945139e-004 2.93165970e-130
  7.12819579e-118 3.47238187e-003 9.61226162e-001]]


Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'


[[1.29899726e-002 9.82213908e-001 4.79611658e-003 2.25026805e-080
  0.00000000e+000 6.39406104e-187 2.67207448e-009]
 [9.93236556e-001 1.90477911e-003 2.12497733e-070 1.05848835e-201
  4.71111066e-291 1.18179566e-076 4.85866472e-003]
 [3.45215395e-002 2.44485078e-004 5.57151955e-012 2.48014517e-115
  9.39372267e-107 2.76920832e-003 9.62464767e-001]
 [2.60201483e-003 8.66855038e-036 5.43757811e-004 2.51346691e-004
  1.08597942e-003 9.89394662e-001 6.12223887e-003]
 [5.08966421e-077 1.23060112e-162 6.30365567e-097 5.02459090e-003
  9.90291321e-001 4.68408846e-003 3.13618393e-028]
 [2.63827761e-075 4.01495058e-075 2.52927168e-004 9.99746833e-001
  2.39802076e-007 1.69762260e-027 5.95374604e-075]
 [1.94083002e-003 7.07357171e-003 9.88014596e-001 2.30107874e-003
  6.76567006e-085 3.37924731e-172 6.69923885e-004]]
[[1.85913171e-003 3.82169479e-003 9.91365670e-001 2.27890973e-003
  2.19261474e-088 6.34181484e-211 6.74593698e-004]
 [1.29109660e-002 9.77942587e-001 9.14644389e-003 1.20410126e-0

In [162]:
sorted(models, key=lambda x: -x[0])[:10]

[(-16668.710820769742,
  CategoricalHMM(init_params='mcs', n_components=7, n_iter=5,
                 random_state=RandomState(MT19937) at 0x7FC19CF09E40)),
 (-16815.088897116242,
  CategoricalHMM(init_params='mcs', n_components=7, n_iter=5,
                 random_state=RandomState(MT19937) at 0x7FC19CF09E40)),
 (-16840.08739415189,
  CategoricalHMM(init_params='mcs', n_components=7, n_iter=5,
                 random_state=RandomState(MT19937) at 0x7FC19CF09E40)),
 (-16926.73251985644,
  CategoricalHMM(init_params='mcs', n_components=7, n_iter=5,
                 random_state=RandomState(MT19937) at 0x7FC19CF09E40)),
 (-17154.062223945613,
  CategoricalHMM(init_params='mcs', n_components=7, n_iter=5,
                 random_state=RandomState(MT19937) at 0x7FC19CF09E40)),
 (-17491.355366015665,
  CategoricalHMM(init_params='mcs', n_components=7, n_iter=5,
                 random_state=RandomState(MT19937) at 0x7FC19CF09E40)),
 (-17605.71244784563,
  CategoricalHMM(init_params='mcs', n_

In [115]:
# make our generative model with two components, a fair die and a
# loaded die
gen_model = hmm.CategoricalHMM(n_components=2, random_state=99)

# the first state is the fair die so let's start there so no one
# catches on right away
gen_model.startprob_ = np.array([1.0, 0.0])

# now let's say that we sneak the loaded die in:
# here, we have a 95% chance to continue using the fair die and a 5%
# chance to switch to the loaded die
# when we enter the loaded die state, we have a 90% chance of staying
# in that state and a 10% chance of leaving
gen_model.transmat_ = np.array([[0.95, 0.05],
                                [0.1, 0.9]])

# now let's set the emission means:
# the first state is a fair die with equal probabilities and the
# second is loaded by being biased toward rolling a six
gen_model.emissionprob_ =
np.array([[1 / 6, 1 / 6, 1 / 6, 1 / 6, 1 / 6, 1 / 6],
          [1 / 10, 1 / 10, 1 / 10, 1 / 10, 1 / 10, 1 / 2]])

In [116]:
gen_model.sample(30)

(array([[2],
        [0],
        [3],
        [0],
        [0],
        [5],
        [4],
        [3],
        [5],
        [4],
        [3],
        [4],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [3],
        [5],
        [0],
        [3],
        [5],
        [5],
        [1],
        [2],
        [5],
        [5],
        [4],
        [0]]),
 array([0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]))

In [269]:
model.decode(data)[1][:100]

array([3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 6, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3,
       3, 4, 5, 5, 5, 6, 6, 6, 0, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5,
       6, 6, 6, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 0, 1,
       1, 2, 2, 2, 3, 3, 4, 4, 5, 5, 5, 5, 6, 6, 6, 0, 0, 1, 2, 2, 2, 3,
       3, 4, 4, 4, 4, 5, 5, 6, 6, 0, 0, 0])